In [1]:
from tensorflow.keras.utils import plot_model

2023-05-28 10:12:50.398625: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-28 10:12:50.420997: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-28 10:12:50.421726: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 10:12:50.976619: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
plot_model(model_full)

NameError: name 'model_full' is not defined

In [3]:
# Imports
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from city_sustainability.models import unet_model
from tensorflow.keras.callbacks import EarlyStopping
from city_sustainability.quality import life_quality
from tensorflow.keras.applications.vgg16 import VGG16
from city_sustainability.preprocessing import image_resize
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from city_sustainability.loading_data_448x448 import loading_paths_448
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from city_sustainability.loading_data_448x448 import image_and_label_arrays_448
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dropout, MaxPooling2D, UpSampling2D, Concatenate, Input

In [4]:
pwd

'/home/marcel/code/Zubairslb/city_sustainability/notebooks/marcel'

In [11]:
# Retrieve image_paths and label_paths
data_path_1 = '../../raw_data/OpenEarthMap_wo_xBD'
image_paths_448, label_paths_448 = loading_paths_448(data_path_1)
# Import all image array as X and label array as y
X, y = image_and_label_arrays_448(image_paths_448, label_paths_448, sampling_ratio=0.1)
# Apply holdout
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
# Scale X and y
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [12]:
X_test_scaled.shape

(81, 448, 448, 3)

In [13]:
# Create the model
model_full = unet_model.build_vgg16_model_full(input_shape=(448, 448, 3), num_classes = 9)

# When running from .py
# model_full = unet_model.build_vgg16_model_full(input_shape=(400, 400, 3), num_classes = 9)

In [14]:
# Compile the model
unet_model.compile_model(model_full)

In [15]:
model_full.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 448, 448, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 448, 448, 64  1792        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 448, 448, 64  36928       ['block1_conv1[0][0]']           
                                )                                                           

                                                                                                  
 batch_normalization_7 (BatchNo  (None, 112, 112, 25  1024       ['conv2d_8[0][0]']               
 rmalization)                   6)                                                                
                                                                                                  
 dropout_7 (Dropout)            (None, 112, 112, 25  0           ['batch_normalization_7[0][0]']  
                                6)                                                                
                                                                                                  
 up_sampling2d_8 (UpSampling2D)  (None, 224, 224, 25  0          ['dropout_7[0][0]']              
                                6)                                                                
                                                                                                  
 concatena

In [16]:
# This line runs the model without batches.
# unet_model.train_model(model_full, X_train_scaled, y_train, epochs = 1, batch_size=18, validation_split=0.2)

# This code below runs the model with batches
# Create an instance of ImageDataGenerator
datagen = ImageDataGenerator(validation_split=0.2)

# Generate train and valid data iterator
train_iterator = datagen.flow(X_train_scaled, y_train, subset='training', batch_size=64)
valid_iterator = datagen.flow(X_train_scaled, y_train, subset='validation', batch_size=64)
steps_per_epoch = len(train_iterator)

# Train the model in batches
epochs = 3
for epoch in range(epochs):
    print("Epoch {}/{}".format(epoch + 1, epochs))
    for step in range(steps_per_epoch):
        X_batch, y_batch = next(train_iterator)
        unet_model.train_model(model_full, X_batch, y_batch)
    
    # Evaluate the model on the validation data
    steps_valid = len(valid_iterator)
    total_val_loss = 0
    for step in range(steps_valid):
        X_val_batch, y_val_batch = next(valid_iterator)
        validation_loss = unet_model.evaluate_model(model_full, X_val_batch, y_val_batch)
        total_val_loss += validation_loss
    print("Average validation loss: {:.4f}".format(total_val_loss/steps_valid))

Epoch 1/3
1/1 [==============================] - 8s 8s/step
Evaluation results:
Loss: 3.5855
Accuracy: 0.3513
Model IoU: 0.2020
Calculated IoU: 0.2020
1/1 [==============================] - 8s 8s/step
Evaluation results:
Loss: 3.3286
Accuracy: 0.3731
Model IoU: 0.2206
Calculated IoU: 0.2206
1/1 [==============================] - 0s 464ms/step
Evaluation results:
Loss: 3.3751
Accuracy: 0.3323
Model IoU: 0.1882
Calculated IoU: 0.1882
Average validation loss: 3.4297
Epoch 2/3
1/1 [==============================] - 8s 8s/step
Evaluation results:
Loss: 4.0549
Accuracy: 0.3542
Model IoU: 0.2147
Calculated IoU: 0.2147


1/1 [==============================] - 8s 8s/step
Evaluation results:
Loss: 3.7387
Accuracy: 0.3627
Model IoU: 0.2211
Calculated IoU: 0.2211
1/1 [==============================] - 1s 537ms/step
Evaluation results:
Loss: 5.0696
Accuracy: 0.2742
Model IoU: 0.1620
Calculated IoU: 0.1620
Average validation loss: 4.2877
Epoch 3/3
1/1 [==============================] - 8s 8s/step
Evaluation results:
Loss: 3.6392
Accuracy: 0.3707
Model IoU: 0.2255
Calculated IoU: 0.2255
1/1 [==============================] - 8s 8s/step
Evaluation results:
Loss: 3.3656
Accuracy: 0.3652
Model IoU: 0.2232
Calculated IoU: 0.2232
1/1 [==============================] - 0s 476ms/step
Evaluation results:
Loss: 4.1693
Accuracy: 0.2783
Model IoU: 0.1590
Calculated IoU: 0.1590
Average validation loss: 3.7247
